In [21]:
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.models import Model,Sequential,model_from_json
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

In [9]:
df_train = pd.read_csv('train_final.csv',index_col=False)
labels = df_train[['784']]
df_train.shape

(156617, 785)

In [10]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
4,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0


In [11]:
np.random.seed(100)

In [12]:
labels = np.array(labels)

In [13]:
cat = to_categorical(labels,num_classes=13)

In [34]:
l = []
for i in range(156617):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))

In [35]:
model = Sequential([
    Conv2D(30,(5,5),input_shape=(28,28,1),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(15,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(50,activation='relu'),
    Dense(13,activation='softmax'),
])

In [36]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 24, 24, 30)        780       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 30)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 15)        4065      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 15)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 5, 15)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 375)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)              

In [38]:
model.fit(np.array(l),cat,epochs=10,batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
784/784 [==============================] - 76s 97ms/step - loss: 0.4076 - accuracy: 0.8953
Epoch 2/10
784/784 [==============================] - 78s 100ms/step - loss: 0.0951 - accuracy: 0.9726
Epoch 3/10
784/784 [==============================] - 78s 100ms/step - loss: 0.0631 - accuracy: 0.9819
Epoch 4/10
784/784 [==============================] - 76s 97ms/step - loss: 0.0483 - accuracy: 0.9861
Epoch 5/10
784/784 [==============================] - 78s 99ms/step - loss: 0.0389 - accuracy: 0.9887
Epoch 6/10
784/784 [==============================] - 88s 112ms/step - loss: 0.0360 - accuracy: 0.9896
Epoch 7/10
784/784 [==============================] - 87s 111ms/step - loss: 0.0297 - accuracy: 0.9911
Epoch 8/10
784/784 [==============================] - 79s 100ms/step - loss: 0.0263 - accuracy: 0.9920
Epoch 9/10
784/784 [==============================] - 78s 100ms/step - loss: 0.0253 - accuracy: 0.9925
Epoch 10/10
784/784 [==============================] - 77s 99ms/step - loss:

In [39]:
model_json = model.to_json()
with open('model_final.json','w') as json_file:
    json_file.write(model_json)
model.save_weights('model_final.h5')